# GreenFlow Rabat — Smart Traffic Intelligence
### Hackathon RamadnIA 2026

**Objective:** Transform existing CCTV cameras into smart traffic sensors using Edge AI.

**Pipeline Overview:**
1. Extract frames from local Rabat driving footage
2. Annotate with Roboflow Label Assist (4 classes)
3. Train YOLO11n with small-object optimizations
4. Export to ONNX/NCNN for Raspberry Pi 5 / Jetson
5. Real-time RTSP inference → JSON webhook → n8n orchestration

**Classes:** `License Plate` · `Car` · `Grand Taxi` · `Triporteur`

**Privacy:** 100% local processing. No images leave the edge device.

## Phase 1: Environment Setup
Installs all dependencies and verifies the hardware (CPU vs GPU).  
Run this cell once per session.

In [9]:
# ---------- Install Dependencies ----------
# We use subprocess instead of !pip to work on ANY platform
# (local, Colab, Kaggle, Paperspace — no Colab-specific syntax).

import subprocess, sys

packages = [
    "ultralytics",            # YOLO11n — our detection model
    "roboflow",               # Dataset download + annotation API
    "opencv-python-headless", # Image/video processing (headless = no GUI needed)
    "requests",               # HTTP calls to n8n webhooks
]

for pkg in packages:
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "-q", pkg],
        stdout=subprocess.DEVNULL  # suppress noisy pip output
    )

print("All packages installed.")

All packages installed.


In [10]:
# ---------- Hardware Check ----------
# This tells us if we have a GPU available.
# Training on CPU = hours.  Training on GPU = minutes.

import torch
import platform

print("=" * 45)
print("  GreenFlow Rabat — Environment Report")
print("=" * 45)
print(f"  Python :  {platform.python_version()}")
print(f"  PyTorch:  {torch.__version__}")
print(f"  CUDA   :  {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  GPU    :  {torch.cuda.get_device_name(0)}")
    print(f"  VRAM   :  {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  GPU    :  None (CPU mode)")
    print("  Tip    :  Connect Colab GPU for training (Phase 5)")

print(f"  OS     :  {platform.system()} {platform.machine()}")
print("=" * 45)

  GreenFlow Rabat — Environment Report
  Python :  3.12.12
  PyTorch:  2.9.0+cu128
  CUDA   :  True
  GPU    :  Tesla T4
  VRAM   :  15.6 GB
  OS     :  Linux x86_64


## Phase 2: Frame Extraction
Extract 1 frame every 5 seconds from the Rabat driving video.  
Output: ~480 JPEG images saved to `data/frames/` for Roboflow annotation.

In [16]:
# ---------- Configuration ----------
# We mount Google Drive to access the video AND save frames there.
# Everything stays on Drive — permanent, survives session restarts.

from pathlib import Path
from google.colab import drive

# Mount Google Drive (a browser popup will ask for permission once)
drive.mount("/content/drive")

# Path to the video on YOUR Drive
VIDEO_PATH = "/content/drive/MyDrive/GreenFlow-Rabat/YTDown.com_YouTube_Driving-in-Rabat-MOROCCO-4K-Car-TOUR-202_Media_jutQ6A_3kPg_001_1080p.mp4"

# Extracted frames saved to your existing Drive folder (permanent storage)
OUTPUT_DIR = Path("/content/drive/MyDrive/GreenFlow-Rabat/Extracted_Frames")
INTERVAL_SEC = 5  # Extract 1 frame every N seconds

# Sanity check
import os
if os.path.exists(VIDEO_PATH):
    size_mb = os.path.getsize(VIDEO_PATH) / 1e6
    print(f"Video found: {VIDEO_PATH} ({size_mb:.0f} MB)")
else:
    print(f"Video NOT found at: {VIDEO_PATH}")
    print("Check the filename in your Drive folder and update VIDEO_PATH above.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Video found: /content/drive/MyDrive/GreenFlow-Rabat/YTDown.com_YouTube_Driving-in-Rabat-MOROCCO-4K-Car-TOUR-202_Media_jutQ6A_3kPg_001_1080p.mp4 (1384 MB)


In [17]:
# ---------- Extract Frames ----------
import cv2

# Create output folder if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

# Read video metadata
fps = cap.get(cv2.CAP_PROP_FPS)                    # Frames per second of the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
duration_min = total_frames / fps / 60              # Video length in minutes
frame_step = int(fps * INTERVAL_SEC)                # How many frames to skip between captures

print(f"Video info: {fps:.1f} FPS | {total_frames:,} total frames | {duration_min:.1f} min")
print(f"Strategy:   1 frame every {INTERVAL_SEC}s = every {frame_step} frames")
print(f"Expected:   ~{total_frames // frame_step} frames to extract\n")

# Main extraction loop
saved = 0
frame_idx = 0

while True:
    ret, frame = cap.read()       # Read one frame from the video
    if not ret:                   # ret is False when video ends
        break
    
    if frame_idx % frame_step == 0:  # Only save every Nth frame
        filename = OUTPUT_DIR / f"rabat_{saved:04d}.jpg"
        cv2.imwrite(str(filename), frame)
        saved += 1
        
        # Progress update every 50 saved frames
        if saved % 50 == 0:
            print(f"  Saved {saved} frames...")
    
    frame_idx += 1

cap.release()  # Always release the video capture when done

print(f"\nDone! Extracted {saved} frames → {OUTPUT_DIR}/")

Video info: 30.0 FPS | 78,338 total frames | 43.5 min
Strategy:   1 frame every 5s = every 150 frames
Expected:   ~522 frames to extract

  Saved 50 frames...
  Saved 100 frames...
  Saved 150 frames...
  Saved 200 frames...
  Saved 250 frames...
  Saved 300 frames...
  Saved 350 frames...
  Saved 400 frames...
  Saved 450 frames...
  Saved 500 frames...

Done! Extracted 523 frames → /content/drive/MyDrive/GreenFlow-Rabat/Extracted_Frames/
